###             Extracting Top 100 Movies of All Time From Rottentomatoes.com

                                    by

                                Jacktone Etemesi

                                phone: +254708578181
                             email: jacktoneetemesi1@gmail.com

The aim of this project is to collect the following details on the top 100 movies of all time from www.rottentomatoes.com;

    1. Movie_index - Refers to the rank of a given movie in the list.
    2. Movie_name - Title of the movie
    3. Rt_rating - Refers to ratings assigned to the movie by rotten tomatoes
    4. Release_Year - Refers to the year the movie started showing
    5. Critics_consesus - Refers to a unified opinion of viewers who criticized the movie
    6. Cast - Actors who took part in the movie
    7. Director - Refers to the individual ho directed the movie
    8. Url - Refers to the link to the movie's view page.
    
Project will utilize the following data analysis Concepst:

    - Web scraping using Beautifulsoup
    - Database 

## Step 1: Scraping Data

In [1]:
#Importing the relevant libraries needed to scrap and store data
import requests
from bs4 import BeautifulSoup,Comment
import sqlite3

In [2]:
# creating a database to store our scrapped data
conn=sqlite3.connect('movies.db')
curs=conn.cursor()
curs.execute('''CREATE TABLE IF NOT EXISTS topmovies (Movie_index INT,Movie_name VARCHAR,Rt_ratings INT,Release_year INT,Critics_consensus VAR,Cast VAR,Director VAR,Url VAR)''')

In [3]:
#Fetching urls containing details on the 100 movies
base_url='https://editorial.rottentomatoes.com/guide/100-best-classic-movies/'
for page_index in range(1,6,1):
    if page_index==1:
        url=base_url
    else:
        url=f'https://editorial.rottentomatoes.com/guide/100-best-classic-movies/{page_index}/'
 #Code to scrap data from the url loop above       
    html=requests.get(url)
    soup=BeautifulSoup(html.text,'lxml')
    master_tree=soup.find_all('div',class_='row countdown-item')
    for master in master_tree:
        movie_index=int(master.find('div',class_='countdown-index').text[1:])
        movie_name=master.find('div',class_='article_movie_title').find('a').text
        rt_rating=int(master.find('div',class_='article_movie_title').find('span',class_='tMeterScore').text[:-1])
        release_year=int(master.find('div',class_='article_movie_title').find('span',class_='subtle start-year').text.replace('(','').replace(')',''))
        critics_consensus=master.find('div',class_='info critics-consensus').contents[1]
        cast=master.find('div',class_='info cast').text.replace('Starring:','').strip()
        director=master.find('div',class_='info director').find('a',class_="").text
        url=master.find('a',class_='article_movie_poster')['href']
        # inserting the scraped data into our database
        curs.execute('''INSERT INTO topmovies (Movie_index,Movie_name,Rt_ratings,Release_year,Critics_consensus,Cast,Director,Url) VALUES (?,?,?,?,?,?,?,?)''',(movie_index,movie_name,rt_rating,release_year,critics_consensus,cast,director,url))
    if page_index<6:
        print(f'Successfully Scraped Data from Page {page_index}')
    conn.commit()
    #print(movie_index,movie_name,rt_rating,release_year,critics_consensus,cast,director)




Successfully Scraped Data from Page 1
Successfully Scraped Data from Page 2
Successfully Scraped Data from Page 3
Successfully Scraped Data from Page 4
Successfully Scraped Data from Page 5


## Step 2: EDA

In [4]:
import numpy as np
import pandas as pd

In [5]:
df=pd.read_sql_query('SELECT * FROM topmovies',conn)
df.drop_duplicates(inplace=True)

In [6]:
df.head()

,Movie_index,Movie_name,Rt_ratings,Release_year,Critics_consensus,Cast,Director,Url
0,100,The Bridge on the River Kwai,96,1957,"This complex war epic asks hard questions, re...","William Holden, Alec Guinness, Sessue Hayakawa...",David Lean,https://www.rottentomatoes.com/m/bridge_on_the...
1,99,Paths of Glory,96,1957,,"Kirk Douglas, Ralph Meeker, Adolphe Menjou, Ge...",Stanley Kubrick,https://www.rottentomatoes.com/m/paths_of_glory
2,98,Rebel Without a Cause,93,1955,,"James Dean, Natalie Wood, Sal Mineo, Jim Backus",Nicholas Ray,https://www.rottentomatoes.com/m/rebel_without...
3,97,A Fistful of Dollars,96,1964,With Akira Kurosawa's,"Clint Eastwood, Marianne Koch, Joe Edger, Wolf...",Sergio Leone,https://www.rottentomatoes.com/m/fistful_of_do...
4,96,The Birds,93,1963,Proving once again that build-up is the key t...,"Rod Taylor, Tippi Hedren, Suzanne Pleshette, J...",Alfred Hitchcock,https://www.rottentomatoes.com/m/1002448-birds


The data has been successfully loaded into a data frame, ready for Cleaning and Analysis